# Frustrated or not

Анализ текстов на фрустрированность

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

In [3]:
df_frustrated = pd.read_excel('/content/drive/MyDrive/research/фрустрация.xlsx')
df_frustrated.head()

,Unnamed: 0,Доля глаголов 1 лица,Доля глаголов 2 лица,Доля глаголов 3 лица,Доля глаголов прошедшего времени,"Доля глаголов прошедшего времени, первого лица, единственного числа",Доля инфинитивов,Доля местоимений 1 лица ед. числа,Доля местоимений 1 лица мн. числа,Доля местоимений 1-ого лица,Доля местоимений 2-ого лица,Доля местоимений 3-ого лица,Коэффициент Трейгера,Коэффициент логической связности,Коэффициент опредмеченности действия (кол. глаголов / кол. существительных),Максимальная глубина синтаксического дерева,Минимальная глубина синтаксического дерева,Причастия + деепричастия / число предложений,Семантическая роль: аблатив,Семантическая роль: абстинатив,Семантическая роль: авторизатор,Семантическая роль: агенс,Семантическая роль: адресат,Семантическая роль: владелец_инф,Семантическая роль: вступление_в_соц_завис,Семантическая роль: высший_предм_иерархии,Семантическая роль: выход_из_соц_завис,Семантическая роль: генератив,Семантическая роль: дейст_в_мс_мод,Семантическая роль: действие_в_фаз_модиф,Семантическая роль: делибератив,Семантическая роль: дестинатив,Семантическая роль: деструктив,Семантическая роль: дименсив,Семантическая роль: директив,Семантическая роль: дистрибутив,Семантическая роль: длительность,Семантическая роль: донор,Семантическая роль: допол_частью_целое,Семантическая роль: замещенный_предмет,...,Часть речи: союз,Часть речи: существительное,Часть речи: частица,Часть речи: часть композита - сложного слова,Часть речи: числительное,Часть речи: числительное-прилагательное,Число знаков препинания / число слов,"Число отрицательных приставок и форм (не,ни,бес,без) / Число слов",Число предложений,Число слов,Число уникальных слов / число слов,словари_Безысключительная и усилительная лексика,словари_Инвективы,словари_Канцеляризмы,словари_Катастрофы,словари_Компьютерный жаргон,"словари_Лексика мотивации, деятельности и напряжения",словари_Лексика отрицательной рациональной оценки,словари_Лексика положительной рациональной оценки и ментальных действий,словари_Лексика положительной эмоциональной оценки,словари_Лексика разрушения и насилия,словари_Лексика сопротивления,словари_Лексика социального разобщения,словари_Лексика страдания,словари_Молодежный жаргон,словари_Мягкие инвективы,словари_Национальные отношения,словари_Обсценная лексика,словари_Отрицательные стеничные эмоции,"словари_Прецедентные, архаизмы, междометия и др.",словари_Социальное неравенство и несправедливость,словари_Тематическая_Власти,словари_Тематическая_Демография и экология,словари_Тематическая_Дети и образование,словари_Тематическая_ЖКХ,словари_Тематическая_Преступность,словари_Тематическая_Протест,словари_Тематическая_Силовые структуры,словари_Тематическая_Экономика,словари_Уголовный жаргон
0,"Автор 095, фрустрационное",0.103611,0.042386,0.185243,0.629513,0.094192,0.276295,0.329640,0.049861,0.379501,0.149584,0.113573,1.305328,0.780022,0.502762,24,2,0.134680,0.001355,0.000000,0.000271,0.000542,0.007319,0.000271,0.0,0.000000,0,0.000000,0.0,0.000271,0.009759,0.003524,0.000000,0.000542,0.005964,0.000271,0.000271,0.002440,0.0,0.0,...,0.093521,0.245595,0.060721,0.0,0.004337,0.004879,0.241800,0.059637,297,3689,0.354297,0.033885,0.011385,0.009217,0.000000,0.005422,0.013283,0.017078,0.178910,0.055842,0.004066,0.002982,0.023313,0.017349,0.007048,0.003253,0.000000,0.001626,0.001626,0.002169,0.001355,0.000271,0.004337,0.001355,0.002169,0.000000,0.000542,0.000271,0.005150,0.003253
1,"Автор 048, фрустрационное",0.029762,0.025298,0.297619,0.717262,0.020833,0.172619,0.110672,0.106719,0.217391,0.051383,0.173913,0.979592,0.793316,0.387543,15,2,0.208443,0.001080,0.000000,0.001080,0.000864,0.004320,0.000000,0.0,0.000216,0,0.000864,0.0,0.000000,0.008423,0.000648,0.000432,0.000216,0.007775,0.000000,0.000000,0.000432,0.0,0.0,...,0.084665,0.319870,0.052268,0.0,0.004320,0.003456,0.222678,0.060043,379,4630,0.392441,0.028294,0.014687,0.034557,0.000000,0.001080,0.007343,0.015119,0.166739,0.049244,0.022030,0.009935,0.027646,0.005832,0.004536,0.003888,0.004536,0.000000,0.0

In [4]:
df_normal = pd.read_excel('/content/drive/MyDrive/research/спокойные.xlsx')
df_normal.head()

,Unnamed: 0,Доля глаголов 1 лица,Доля глаголов 2 лица,Доля глаголов 3 лица,Доля глаголов прошедшего времени,"Доля глаголов прошедшего времени, первого лица, единственного числа",Доля инфинитивов,Доля местоимений 1 лица ед. числа,Доля местоимений 1 лица мн. числа,Доля местоимений 1-ого лица,Доля местоимений 2-ого лица,Доля местоимений 3-ого лица,Коэффициент Трейгера,Коэффициент логической связности,Коэффициент опредмеченности действия (кол. глаголов / кол. существительных),Максимальная глубина синтаксического дерева,Минимальная глубина синтаксического дерева,Причастия + деепричастия / число предложений,Семантическая роль: аблатив,Семантическая роль: абстинатив,Семантическая роль: авторизатор,Семантическая роль: агенс,Семантическая роль: адресат,Семантическая роль: владелец_инф,Семантическая роль: вступление_в_соц_завис,Семантическая роль: высший_предм_иерархии,Семантическая роль: выход_из_соц_завис,Семантическая роль: генератив,Семантическая роль: дейст_в_мс_мод,Семантическая роль: действие_в_фаз_модиф,Семантическая роль: делибератив,Семантическая роль: дестинатив,Семантическая роль: деструктив,Семантическая роль: дименсив,Семантическая роль: директив,Семантическая роль: дистрибутив,Семантическая роль: длительность,Семантическая роль: донор,Семантическая роль: допол_частью_целое,Семантическая роль: замещенный_предмет,...,Часть речи: союз,Часть речи: существительное,Часть речи: частица,Часть речи: часть композита - сложного слова,Часть речи: числительное,Часть речи: числительное-прилагательное,Число знаков препинания / число слов,"Число отрицательных приставок и форм (не,ни,бес,без) / Число слов",Число предложений,Число слов,Число уникальных слов / число слов,словари_Безысключительная и усилительная лексика,словари_Инвективы,словари_Канцеляризмы,словари_Катастрофы,словари_Компьютерный жаргон,"словари_Лексика мотивации, деятельности и напряжения",словари_Лексика отрицательной рациональной оценки,словари_Лексика положительной рациональной оценки и ментальных действий,словари_Лексика положительной эмоциональной оценки,словари_Лексика разрушения и насилия,словари_Лексика сопротивления,словари_Лексика социального разобщения,словари_Лексика страдания,словари_Молодежный жаргон,словари_Мягкие инвективы,словари_Национальные отношения,словари_Обсценная лексика,словари_Отрицательные стеничные эмоции,"словари_Прецедентные, архаизмы, междометия и др.",словари_Социальное неравенство и несправедливость,словари_Тематическая_Власти,словари_Тематическая_Демография и экология,словари_Тематическая_Дети и образование,словари_Тематическая_ЖКХ,словари_Тематическая_Преступность,словари_Тематическая_Протест,словари_Тематическая_Силовые структуры,словари_Тематическая_Экономика,словари_Уголовный жаргон
0,"Автор 084, спокойное",0.075912,0.016058,0.233577,0.636496,0.071533,0.300730,0.405229,0.022876,0.428105,0.016340,0.081699,1.282772,0.712615,0.447712,11,2,0.092732,0.001882,0.000941,0.001411,0.000470,0.003058,0.0,0.0,0.0,0,0.000470,0.0,0.0,0.007998,0.003058,0.000000,0.000000,0.007763,0.000000,0.000000,0.000470,0.0,0.0,...,0.087038,0.287932,0.051753,0.000000,0.008939,0.002823,0.242766,0.055752,399,4251,0.357328,0.034345,0.017408,0.011997,0.000000,0.003058,0.013409,0.016937,0.178546,0.052693,0.005881,0.000235,0.016702,0.016937,0.009880,0.003529,0.000235,0.007057,0.000235,0.003764,0.002117,0.000941,0.001411,0.004705,0.001882,0.000235,0.000000,0.000235,0.000470,0.005881
1,"Автор 027, спокойные",0.040179,0.044643,0.312500,0.698661,0.037946,0.167411,0.200000,0.033333,0.233333,0.088889,0.150000,1.261972,0.922840,0.395760,14,2,0.236111,0.001705,0.001705,0.000682,0.002728,0.001705,0.0,0.0,0.0,0,0.000341,0.0,0.0,0.017729,0.001364,0.001023,0.000000,0.005796,0.000682,0.000000,0.001023,0.0,0.0,...,0.093761,0.324582,0.053529,0.000000,0.003069,0.002046,0.245823,0.041596,216,2933,0.429594,0.016365,0.010569,0.036481,0.000000,0.002387,0.012274,0.009547,0.171838,0.041255,0.021139,0.003069,0.023866,0.004091,0.005114,0.003750,0.000682,0.000341,0.000341,0.014320,0.001364,0.0040

In [5]:
df_frustrated['target'] = 1
df_normal['target'] = 0

df = pd.concat([df_frustrated, df_normal], ignore_index=True)

In [6]:
df.head()

,Unnamed: 0,Доля глаголов 1 лица,Доля глаголов 2 лица,Доля глаголов 3 лица,Доля глаголов прошедшего времени,"Доля глаголов прошедшего времени, первого лица, единственного числа",Доля инфинитивов,Доля местоимений 1 лица ед. числа,Доля местоимений 1 лица мн. числа,Доля местоимений 1-ого лица,Доля местоимений 2-ого лица,Доля местоимений 3-ого лица,Коэффициент Трейгера,Коэффициент логической связности,Коэффициент опредмеченности действия (кол. глаголов / кол. существительных),Максимальная глубина синтаксического дерева,Минимальная глубина синтаксического дерева,Причастия + деепричастия / число предложений,Семантическая роль: аблатив,Семантическая роль: абстинатив,Семантическая роль: авторизатор,Семантическая роль: агенс,Семантическая роль: адресат,Семантическая роль: владелец_инф,Семантическая роль: вступление_в_соц_завис,Семантическая роль: высший_предм_иерархии,Семантическая роль: выход_из_соц_завис,Семантическая роль: генератив,Семантическая роль: дейст_в_мс_мод,Семантическая роль: действие_в_фаз_модиф,Семантическая роль: делибератив,Семантическая роль: дестинатив,Семантическая роль: деструктив,Семантическая роль: дименсив,Семантическая роль: директив,Семантическая роль: дистрибутив,Семантическая роль: длительность,Семантическая роль: донор,Семантическая роль: допол_частью_целое,Семантическая роль: замещенный_предмет,...,Часть речи: существительное,Часть речи: частица,Часть речи: часть композита - сложного слова,Часть речи: числительное,Часть речи: числительное-прилагательное,Число знаков препинания / число слов,"Число отрицательных приставок и форм (не,ни,бес,без) / Число слов",Число предложений,Число слов,Число уникальных слов / число слов,словари_Безысключительная и усилительная лексика,словари_Инвективы,словари_Канцеляризмы,словари_Катастрофы,словари_Компьютерный жаргон,"словари_Лексика мотивации, деятельности и напряжения",словари_Лексика отрицательной рациональной оценки,словари_Лексика положительной рациональной оценки и ментальных действий,словари_Лексика положительной эмоциональной оценки,словари_Лексика разрушения и насилия,словари_Лексика сопротивления,словари_Лексика социального разобщения,словари_Лексика страдания,словари_Молодежный жаргон,словари_Мягкие инвективы,словари_Национальные отношения,словари_Обсценная лексика,словари_Отрицательные стеничные эмоции,"словари_Прецедентные, архаизмы, междометия и др.",словари_Социальное неравенство и несправедливость,словари_Тематическая_Власти,словари_Тематическая_Демография и экология,словари_Тематическая_Дети и образование,словари_Тематическая_ЖКХ,словари_Тематическая_Преступность,словари_Тематическая_Протест,словари_Тематическая_Силовые структуры,словари_Тематическая_Экономика,словари_Уголовный жаргон,target
0,"Автор 095, фрустрационное",0.103611,0.042386,0.185243,0.629513,0.094192,0.276295,0.329640,0.049861,0.379501,0.149584,0.113573,1.305328,0.780022,0.502762,24,2,0.134680,0.001355,0.000000,0.000271,0.000542,0.007319,0.000271,0.0,0.000000,0,0.000000,0.0,0.000271,0.009759,0.003524,0.000000,0.000542,0.005964,0.000271,0.000271,0.002440,0.0,0.0,...,0.245595,0.060721,0.0,0.004337,0.004879,0.241800,0.059637,297,3689,0.354297,0.033885,0.011385,0.009217,0.000000,0.005422,0.013283,0.017078,0.178910,0.055842,0.004066,0.002982,0.023313,0.017349,0.007048,0.003253,0.000000,0.001626,0.001626,0.002169,0.001355,0.000271,0.004337,0.001355,0.002169,0.000000,0.000542,0.000271,0.005150,0.003253,1
1,"Автор 048, фрустрационное",0.029762,0.025298,0.297619,0.717262,0.020833,0.172619,0.110672,0.106719,0.217391,0.051383,0.173913,0.979592,0.793316,0.387543,15,2,0.208443,0.001080,0.000000,0.001080,0.000864,0.004320,0.000000,0.0,0.000216,0,0.000864,0.0,0.000000,0.008423,0.000648,0.000432,0.000216,0.007775,0.000000,0.000000,0.000432,0.0,0.0,...,0.319870,0.052268,0.0,0.004320,0.003456,0.222678,0.060043,379,4630,0.392441,0.028294,0.014687,0.034557,0.000000,0.001080,0.007343,0.015119,0.166739,0.049244,0.022030,0.009935,0.027646,0.005832,0.004536,0.003888,0.004536,0.000000,0.002160,0.009287,0.003456,0.

-------

In [7]:
def cross_validation(model, X, y, n_folds=5, random_state=42, shuffle=True):
    
    X = np.array(X)
    y = np.array(y)
    
    acc_scores = []
    f1_scores = []
    
    skf = StratifiedKFold(n_splits=n_folds, random_state=random_state, shuffle=shuffle)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        acc_scores.append(accuracy_score(y_test, y_pred))
        f1_scores.append(f1_score(y_test, y_pred))
    
    print('accuracy \t {:.2f} +- {:.2f}'.format(np.mean(acc_scores), np.std(acc_scores)))
    print('f1 score \t {:.2f} +- {:.2f}'.format(np.mean(f1_scores), np.std(f1_scores)))

## С бинаризацией



## Все признаки

In [8]:
y = df['target']
X = df.drop(['target', 'Unnamed: 0'], axis=1)
cols = list(X.columns)

In [9]:
# бинаризация
for col in cols:
    q = X[col].quantile(0.7)
    if q <= 0.1:
        X.loc[X[col] >= q, col] = 1
        X.loc[X[col] < q, col] = 0

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42, stratify=y)

### SVM

In [29]:
svm = SVC(kernel='linear',
          C=1)

In [30]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.63 +- 0.04
f1 score 	 0.63 +- 0.05


### Random Forest

In [39]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='entropy',
                             min_samples_split=10,
                             random_state=42)

In [40]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.73 +- 0.07
f1 score 	 0.74 +- 0.06


### LogReg

In [83]:
lrc = LogisticRegression(penalty='l1',
                         solver='liblinear',
                         C=0.4,
                         max_iter=1000)

In [84]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.69 +- 0.07
f1 score 	 0.69 +- 0.08


----------
## Признаки SD (Тематические словари, 9 признаков)

In [85]:
sd_cols = list(filter(lambda s: s.startswith('словари_Тематическая'), cols))

In [86]:
sd_cols

['словари_Тематическая_Власти',
 'словари_Тематическая_Демография и экология',
 'словари_Тематическая_Дети и образование',
 'словари_Тематическая_ЖКХ',
 'словари_Тематическая_Преступность',
 'словари_Тематическая_Протест',
 'словари_Тематическая_Силовые структуры',
 'словари_Тематическая_Экономика']

In [87]:
y = df['target']
X = df[sd_cols]

### SVM

In [118]:
svm = SVC(kernel='rbf',
          C=2,
          probability=False)

In [119]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.58 +- 0.02
f1 score 	 0.56 +- 0.04


### Random Forest

In [136]:
rfc = RandomForestClassifier(n_estimators=200,
                             criterion='gini',
                             min_samples_split=30,
                             random_state=42)

In [137]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.60 +- 0.07
f1 score 	 0.60 +- 0.06


### LogReg

In [163]:
lrc = LogisticRegression(penalty='l2',
                         solver='newton-cg',
                         C=5,
                         max_iter=1000)

In [164]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.51 +- 0.05
f1 score 	 0.36 +- 0.25


----
## Признаки PS (Части речи, 14 признаков)

In [165]:
ps_cols = list(filter(lambda s: s.startswith('Часть речи'), cols))
ps_cols

['Часть речи: глагол',
 'Часть речи: междометие',
 'Часть речи: местоимение-прилагательное',
 'Часть речи: местоимение-существительное',
 'Часть речи: местоименное наречие',
 'Часть речи: наречие',
 'Часть речи: предлог',
 'Часть речи: прилагательное',
 'Часть речи: союз',
 'Часть речи: существительное',
 'Часть речи: частица',
 'Часть речи: часть композита - сложного слова',
 'Часть речи: числительное',
 'Часть речи: числительное-прилагательное']

In [166]:
y = df['target']
X = df[ps_cols]

### SVM

In [186]:
svm = SVC(kernel='rbf',
          C=30,
          probability=False)

In [187]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.61 +- 0.06
f1 score 	 0.61 +- 0.04


### Random Forest

In [202]:
rfc = RandomForestClassifier(n_estimators=600,
                             criterion='entropy',
                             min_samples_split=30,
                             random_state=42)

In [203]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.59 +- 0.04
f1 score 	 0.60 +- 0.04


### LogReg

In [220]:
lrc = LogisticRegression(penalty='l1',
                         solver='liblinear',
                         C=70,
                         max_iter=1000)

In [221]:
cross_validation(lrc, X, y, n_folds=5)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


accuracy 	 0.63 +- 0.04
f1 score 	 0.64 +- 0.02


---
## Признаки AS (Cловари оценки и состояния, 20 признаков)

In [222]:
as_cols = list(filter(lambda s: s.startswith('словари') and not s.startswith('словари_Тематическая'),
                     cols))
as_cols

['словари_Безысключительная и усилительная лексика',
 'словари_Инвективы',
 'словари_Канцеляризмы',
 'словари_Катастрофы',
 'словари_Компьютерный жаргон',
 'словари_Лексика мотивации, деятельности и напряжения',
 'словари_Лексика отрицательной рациональной оценки',
 'словари_Лексика положительной рациональной оценки и ментальных действий',
 'словари_Лексика положительной эмоциональной оценки',
 'словари_Лексика разрушения и насилия',
 'словари_Лексика сопротивления',
 'словари_Лексика социального разобщения',
 'словари_Лексика страдания',
 'словари_Молодежный жаргон',
 'словари_Мягкие инвективы',
 'словари_Национальные отношения',
 'словари_Обсценная лексика',
 'словари_Отрицательные стеничные эмоции',
 'словари_Прецедентные, архаизмы, междометия и др.',
 'словари_Социальное неравенство и несправедливость',
 'словари_Уголовный жаргон']

In [223]:
y = df['target']
X = df[as_cols]

### SVM

In [247]:
svm = SVC(kernel='rbf',
          C=20,
          probability=False)

In [248]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.60 +- 0.05
f1 score 	 0.61 +- 0.06


### Random Forest

In [255]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='gini',
                             min_samples_split=20,
                             random_state=42)

In [256]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.65 +- 0.07
f1 score 	 0.65 +- 0.08


### LogReg

In [271]:
lrc = LogisticRegression(penalty='l2',
                         solver='newton-cg',
                         C=30,
                         max_iter=1000)

In [272]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.61 +- 0.05
f1 score 	 0.63 +- 0.05


---
## Признаки SR (Семантические роли, 92 признака)

In [273]:
sr_cols = list(filter(lambda s: s.startswith('Семантическая роль'), cols))
sr_cols

['Семантическая роль: аблатив',
 'Семантическая роль: абстинатив',
 'Семантическая роль: авторизатор',
 'Семантическая роль: агенс',
 'Семантическая роль: адресат',
 'Семантическая роль: владелец_инф',
 'Семантическая роль: вступление_в_соц_завис',
 'Семантическая роль: высший_предм_иерархии',
 'Семантическая роль: выход_из_соц_завис',
 'Семантическая роль: генератив',
 'Семантическая роль: дейст_в_мс_мод',
 'Семантическая роль: действие_в_фаз_модиф',
 'Семантическая роль: делибератив',
 'Семантическая роль: дестинатив',
 'Семантическая роль: деструктив',
 'Семантическая роль: дименсив',
 'Семантическая роль: директив',
 'Семантическая роль: дистрибутив',
 'Семантическая роль: длительность',
 'Семантическая роль: донор',
 'Семантическая роль: допол_частью_целое',
 'Семантическая роль: замещенный_предмет',
 'Семантическая роль: инструментив',
 'Семантическая роль: интенсив',
 'Семантическая роль: интерпрет_понятия',
 'Семантическая роль: источник_кауз_отн',
 'Семантическая роль: исходно

In [274]:
y = df['target']
X = df[sr_cols]

### SVM

In [294]:
svm = SVC(kernel='rbf',
          C=30,
          probability=False)

In [295]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.60 +- 0.08
f1 score 	 0.60 +- 0.11


### Random Forest

In [300]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='gini',
                             min_samples_split=30,
                             random_state=42)

In [301]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.65 +- 0.07
f1 score 	 0.65 +- 0.07


### LogReg

In [332]:
lrc = LogisticRegression(penalty='l2',
                         solver='newton-cg',
                         C=90,
                         max_iter=1000)

In [333]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.56 +- 0.03
f1 score 	 0.56 +- 0.05


---
## Признаки SL (Семантические связи, 35 признаков)

In [334]:
sl_cols = list(filter(lambda s: s.startswith('Семантическая связь'), cols))
sl_cols

['Семантическая связь:ABL',
 'Семантическая связь:ABS',
 'Семантическая связь:ADR',
 'Семантическая связь:CAUS',
 'Семантическая связь:CMP',
 'Семантическая связь:COM',
 'Семантическая связь:COR',
 'Семантическая связь:DES',
 'Семантическая связь:DIR',
 'Семантическая связь:DIS',
 'Семантическая связь:DLB',
 'Семантическая связь:DST',
 'Семантическая связь:EQ',
 'Семантическая связь:FAB',
 'Семантическая связь:GEN',
 'Семантическая связь:INS',
 'Семантическая связь:LIM',
 'Семантическая связь:LIQ',
 'Семантическая связь:LOC',
 'Семантическая связь:MED',
 'Семантическая связь:OBJ',
 'Семантическая связь:PAR',
 'Семантическая связь:POS',
 'Семантическая связь:POT',
 'Семантическая связь:PRM',
 'Семантическая связь:PTN',
 'Семантическая связь:QLT',
 'Семантическая связь:QNT',
 'Семантическая связь:RSN',
 'Семантическая связь:SIT',
 'Семантическая связь:SUR',
 'Семантическая связь:TAR',
 'Семантическая связь:TMP',
 'Семантическая связь:TRA',
 'Семантическая связь:TRG']

In [335]:
y = df['target']
X = df[sl_cols]

### SVM

In [351]:
svm = SVC(kernel='rbf',
          C=10,
          probability=False)

In [352]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.60 +- 0.04
f1 score 	 0.60 +- 0.06


### Random Forest

In [369]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='gini',
                             min_samples_split=30,
                             random_state=42)

In [370]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.59 +- 0.04
f1 score 	 0.59 +- 0.07


### LogReg

In [385]:
lrc = LogisticRegression(penalty='l2',
                         solver='lbfgs',
                         C=30,
                         max_iter=1000)

In [386]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.55 +- 0.06
f1 score 	 0.50 +- 0.25


---
## Признаки PI (Психолингвистические показатели, 27 признаков)

In [387]:
other_cols = set.union(set(sd_cols), set(ps_cols), set(as_cols), set(sr_cols), set(sl_cols))
pi_cols = list(set(cols).difference(other_cols))
# pi_cols.remove('target')
# pi_cols.remove('Unnamed: 0')
pi_cols

['Средняя глубина синтаксического дерева',
 'Средняя длина слов',
 'Доля местоимений 2-ого лица',
 'Число уникальных слов / число слов',
 'Число знаков препинания / число слов',
 'Средняя длина предложений (в количестве слов)',
 'Среднее число клауз в предложении',
 'Доля глаголов прошедшего времени',
 'Число предложений',
 'Причастия + деепричастия / число предложений',
 'Коэффициент Трейгера',
 'Доля глаголов 3 лица',
 'Минимальная глубина синтаксического дерева',
 'Доля местоимений 1-ого лица',
 'Доля глаголов прошедшего времени, первого лица, единственного числа',
 'Доля инфинитивов',
 'Доля местоимений 1 лица мн. числа',
 'Тональность слов',
 'Доля местоимений 3-ого лица',
 'Доля глаголов 1 лица',
 'Коэффициент опредмеченности действия (кол. глаголов / кол. существительных)',
 'Коэффициент логической связности',
 'Максимальная глубина синтаксического дерева',
 'Доля глаголов 2 лица',
 'Число отрицательных приставок и форм (не,ни,бес,без) / Число слов',
 'Число слов',
 'Доля местои

In [388]:
y = df['target']
X = df[pi_cols]

### SVM

In [403]:
svm = SVC(kernel='linear',
          C=1,
          probability=False)

In [404]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.75 +- 0.05
f1 score 	 0.73 +- 0.06


### Random Forest

In [416]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='entropy',
                             min_samples_split=30,
                             random_state=42)

In [417]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.74 +- 0.04
f1 score 	 0.74 +- 0.04


### LogReg

In [454]:
lrc = LogisticRegression(penalty='l1',
                         solver='liblinear',
                         C=5,
                         max_iter=1000)

In [455]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.76 +- 0.05
f1 score 	 0.77 +- 0.06


--------

In [457]:
top_cols = ['Семантическая роль: деструктив', 'Семантическая роль: ликвидатив',
            'Тональность слов', 'Число отрицательных приставок и форм (не,ни,бес,без) / Число слов',
            'Часть речи: частица', 'Семантическая роль: каузатив',
            'словари_Инвективы', 'Число знаков препинания / число слов',
            'Доля местоимений 1-ого лица', 'словари_Лексика сопротивления']

In [459]:
X = df[top_cols]

### SVM

In [518]:
svm = SVC(kernel='linear',
          C=50)

In [519]:
cross_validation(svm, X, y, n_folds=5)

accuracy 	 0.78 +- 0.03
f1 score 	 0.77 +- 0.03


### Random Forest

In [533]:
rfc = RandomForestClassifier(n_estimators=500,
                             criterion='entropy',
                             min_samples_split=20,
                             random_state=42)

In [534]:
cross_validation(rfc, X, y, n_folds=5)

accuracy 	 0.77 +- 0.06
f1 score 	 0.77 +- 0.07


### LogReg

In [573]:
lrc = LogisticRegression(penalty='l1', 
                         solver='liblinear',
                         C=50,
                         max_iter=1000)

In [574]:
cross_validation(lrc, X, y, n_folds=5)

accuracy 	 0.79 +- 0.07
f1 score 	 0.80 +- 0.07
